In [2]:
import dypac as dp
import numpy as np
import sklearn as sk
import scipy as sp
import matplotlib.pyplot as plt
import time
from nilearn import datasets
from nilearn import image
from nilearn.input_data import NiftiMasker
from nilearn import plotting
from nilearn.decomposition import DictLearning, CanICA
from scipy.stats import pearsonr

In [5]:
dp.

AttributeError: module 'dypac' has no attribute 'tests'

In [ ]:
n_subjects = 10
adhd_dataset = datasets.fetch_adhd(n_subjects)
epi_filename = adhd_dataset.func[0]

In [ ]:
import pickle
hf =  open('model.pickle','rb')
model = pickle.load(hf)

In [ ]:
model.components_.shape

In [ ]:
model.components_ = model.components_.todense()

In [ ]:
tseries = model.masker_.transform(epi_filename)

In [ ]:
from dypac import Embedding

In [ ]:
tseries.shape

In [ ]:
weights = model.transform_sparse(epi_filename)

In [ ]:
num_comp = 15
plt.plot(weights[:, num_comp])

In [ ]:
num_t = 50
img = model.masker_.inverse_transform(tseries[num_t,:])
plotting.view_img(img, cut_coords=[2, -22, 10], vmax=4)

In [ ]:
img_r = model.masker_.inverse_transform(weights[num_t,:] * model.components_)
plotting.view_img(img_r, cut_coords=[2, -22, 10], vmax=4)

In [ ]:
img_diff = image.new_img_like(img, img.get_fdata() - np.squeeze(img_r.get_fdata()))
plotting.view_img(img_diff, cut_coords=[2, -22, 10], vmax=4)

Let's calculate a std image of the residuals:

In [ ]:
for tt in range(0,tseries.shape[0]):
    img = model.masker_.inverse_transform(tseries[tt, :])
    img_r = model.masker_.inverse_transform(weights[tt,:] * model.components_)
    if (tt==0):
        diff = np.square(img.get_fdata() - np.squeeze(img_r.get_fdata()))
    else:
        diff = diff + np.square(img.get_fdata() - np.squeeze(img_r.get_fdata()))
img_std = image.new_img_like(img, diff / tseries.shape[0])
plotting.view_img(img_std, cut_coords=[2, -22, 10], vmax=1)